In [ ]:
from IPython.display import display, Image
import json
import os
from confluent_kafka import Consumer, KafkaError

# Thông tin cấu hình Kafka
kafka_bootstrap_servers = 'localhost:9092'
kafka_topic = 'videodetect'

# Cấu hình consumer
consumer_conf = {
    'bootstrap.servers': kafka_bootstrap_servers,
    'group.id': 'my_consumer_group',
    'auto.offset.reset': 'earliest'
}

# Khởi tạo consumer
consumer = Consumer(consumer_conf)

# Subscribe vào topic
consumer.subscribe([kafka_topic])

# Khởi tạo biến để theo dõi sự thay đổi
previous_files = set()

try:
    while True:
        msg = consumer.poll(timeout=1000)  # Poll for messages, waiting at most 1 second

        if msg is None:
            continue
        if msg.error():
            if msg.error().code() == KafkaError._PARTITION_EOF:
                # End of partition event, not an error
                continue
            else:
                print(msg.error())
                break

        # Lấy dữ liệu từ message
        event_data = json.loads(msg.value())
        frame_number = event_data.get('frame', -1)
        image_path = f"C:\\Users\\admin\\PycharmProjects\\BigData\\CSVanh\\frame_{frame_number}.jpg"

        # Kiểm tra xem tệp có tồn tại không
        if os.path.exists(image_path):
            # Lấy danh sách các tệp hiện tại trong thư mục
            current_files = {image_path}

            # Tìm sự khác biệt giữa tệp hiện tại và tệp trước đó
            new_files = current_files - previous_files

            # Lặp qua tất cả các tệp mới và hiển thị chúng
            for new_file in new_files:
                # Hiển thị hình ảnh trên console Jupyter
                display(Image(filename=new_file))

                # In tên tệp ra console
                print(f"Tệp ảnh: {new_file}")

            # Cập nhật tệp trước đó để sử dụng trong lần lặp tiếp theo
            previous_files = current_files

except KeyboardInterrupt:
    pass

finally:
    # Đóng consumer khi kết thúc
    consumer.close()
